# MODULOS

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

# Creacion Data Frames

In [2]:
#Importo y creo todos los Data sets
df_ALUCarrera = pd.read_csv('Alu_Carrera.csv', sep=';')
df_AluMateria = pd.read_csv('Alu_Materia.csv', sep=';')
df_Carreras = pd.read_csv('Carreras.csv', sep=';')
df_Finales = pd.read_csv('Finales.csv', sep=';')
df_Materias = pd.read_csv('Materias.csv', sep=';')

# LIMPIEZA

### Tranformo el tipo de dato tiempo en los DF

In [3]:
#Arreglo el problema de que los datos de tipo tiempo estan como si fueran tipo: objeto en vez de tipo: DataTime
df_ALUCarrera[['Fecha_Ingreso','Fecha_Nac']] = df_ALUCarrera[['Fecha_Ingreso','Fecha_Nac']].apply(pd.to_datetime)
df_AluMateria[['Fecha_Inscrip','Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup']] = df_AluMateria[['Fecha_Inscrip','Fecha_1p','Fecha_2p','Fecha_TP','Fecha_Recup']].apply(pd.to_datetime)
df_Finales[['Fecha_Examen','Fecha_Inscrip_Examen']] = df_Finales[['Fecha_Examen','Fecha_Inscrip_Examen']].apply(pd.to_datetime)

### Eliminacion de Carreras

In [4]:
# veo cuales carreras no tienen datos suficientes para merecer su analisis.
df_ALUCarrera['Id_Carrera'].value_counts()

5     963
6     687
7     676
2     641
4     616
1     582
3     506
9     494
13    334
10    270
8     204
17    127
14     57
60     37
16     14
39      8
18      3
11      2
12      2
40      2
41      1
59      1
Name: Id_Carrera, dtype: int64

In [5]:
#Averiguo como se llaman las carreras
listaDeCarreras = [11,60,18,39,59,40,41,16,12]
df_Carreras[df_Carreras['Id_Carrera'].isin(listaDeCarreras)]

,Id_Carrera,Nombre_Largo,Nombre_Corto,Abreviatura,Cuatrimestres
2,12,ANALISTA EN GESTION DE VENTAS,VENTAS,VTA,4
6,18,TECNICO SUPERIOR EN LOGISTICA,LOGÍSTICA,LOG,6
8,41,AGENTE DE TRANSPORTE ADUANERO Y DESPACHANTE DE...,AGENTE DE TRANSP. ADUANERO Y DESP. DE ADUANA,ATA,4
12,59,TÉCNICO SUPERIOR EN SERVICIOS DE PAISAJISMO,SERVICIOS DE PAISAJISMO,SERPAI,2
13,39,ADMINISTRADOR DE REDES,ADMINISTRADOR DE REDES,AR,4
14,16,TECNICO SUPERIOR EN GESTION JURIDICA,GESTIÓN JURíDICA,GJ,6
17,60,TÉC. SUP. EN DES. DE SIMULACIONES VIRTUALES Y ...,VIDEOJUEGOS,SVIRT,6
18,40,TÉCNICO EN SERVICIOS AGROPECUARIOS,TÉCNICO EN SERVICIOS AGROPECUARIOS,TSAGR,4
19,11,ASISTENTE GERENCIAL TRILINGÜE,TRILINGÜE,TRI,4


In [6]:
#Averiguo los alumnos que estan en cada carrera y en total son solo 70 datos los que voy a eliminar, lo cual me parece irrelevante.
df_ALUCarrera.loc[df_ALUCarrera['Id_Carrera'].isin(listaDeCarreras)].count()
#al mismo tiempo creo que tener 9 carreras mas, que no me aportarian valor a mi analisis si es molesto.

Id_Alumno           70
Id_Carrera          70
Semestre_Ingreso    70
Año_Ingreso         70
Id_Ctro_Ext         70
Id_Modalidad        70
Fecha_Ingreso       70
Activo              70
Sexo                70
Fecha_Nac           70
dtype: int64

In [7]:
#Elimino todo registro de las carreras que tienen datos ignorables
df_Carreras = df_Carreras[~df_Carreras['Id_Carrera'].isin(listaDeCarreras)]
df_Finales = df_Finales[~df_Finales['Id_Carrera'].isin(listaDeCarreras)]
df_ALUCarrera = df_ALUCarrera[~df_ALUCarrera['Id_Carrera'].isin(listaDeCarreras)]
df_AluMateria = df_AluMateria[~df_AluMateria['Id_Carrera'].isin(listaDeCarreras)]

### Elimino Años que no me interesan de el DF AluMateria

In [8]:
df_AluMateria['Año'].describe()

count    130673.000000
mean       2007.038654
std           3.238610
min        1982.000000
25%        2005.000000
50%        2007.000000
75%        2009.000000
max        2021.000000
Name: Año, dtype: float64

In [9]:
# Vacio los años que no esten entre 2001 y 2010 ya que si los graficamos hay un monton de datos faltantes en varias carreras. 
df_AluMateria.loc[(df_AluMateria['Año']<2001) | (df_AluMateria['Año']>2010),['Fecha_Inscrip','Año']]=np.nan

In [10]:
#Verifico
df_AluMateria['Año'].unique()

array([2006., 2005., 2007., 2001., 2002., 2004., 2003., 2009., 2010.,
         nan, 2008.])

### Elimino los datos que no sean de la sucursal de cordoba

### Columna Edad 

In [11]:
#Busco Vaciar todos los datos de la fecha de nacimiento que son un error ya que nadie menor a 17 podria estar cursando la carrera 
df_ALUCarrera.loc[(pd.DatetimeIndex(df_ALUCarrera['Fecha_Ingreso']).year - pd.DatetimeIndex(df_ALUCarrera['Fecha_Nac']).year)<17, 'Fecha_Nac'] = np.nan

In [12]:
#Creo una columna llamada edad 
df_ALUCarrera['Edad'] = pd.DatetimeIndex(df_ALUCarrera['Fecha_Ingreso']).year - pd.DatetimeIndex(df_ALUCarrera['Fecha_Nac']).year 

In [13]:
#Verifico que los cambios se hayan hecho bien
df_ALUCarrera['Edad'].unique()

array([19., 52., 48., 43., 20., 21., 38., 39., 37., 34., 35., 36., 23.,
       22., 26., 25., 18., 33., 41., 32., 31., 40., 30., 29., 28., 27.,
       24., nan, 17., 53.])

### Sucursal Cordoba

In [14]:
df_ALUCarrera['Id_Ctro_Ext'].value_counts()

CO    6146
SF       4
VA       2
VM       2
JM       2
AR       1
Name: Id_Ctro_Ext, dtype: int64

In [15]:
#Veo cuantos datos hay de cordoba
df_AluMateria['Ctro_Ext'].value_counts()

CO    129165
JM       851
VM       206
VA       139
SF       100
CP        90
RC        69
AR        32
HE        13
VD         8
Name: Ctro_Ext, dtype: int64

In [16]:
#Elimino todo lo que no sea de cordoba
df_AluMateria = df_AluMateria[df_AluMateria['Ctro_Ext']=='CO']
df_ALUCarrera = df_ALUCarrera[df_ALUCarrera['Id_Ctro_Ext']=='CO']

# Manipulacion y modificacion de datos

### Estado de alumno en la materia

Estado del alumno en la materia: vasicamente quiero revisar en que condicion se encuentra el alumno en cada fila de la materia de la cual tengo un dato sobre el quiero saber si es regular, promociono la materia o esta libre en la misma.

In [17]:
#Creo la columna y la lleno con un string arbitrario para poder modificarla luego
df_AluMateria['Condicion'] = 'Estado'

In [18]:
#Creo la condicion de promocionado.
df_AluMateria.loc[  (df_AluMateria['Nota_1p'] >= 7 ) & (df_AluMateria['Nota_1p'] != 11) 
                    & (df_AluMateria['Nota_2p'] >= 7 ) & (df_AluMateria['Nota_2p'] != 11)
                    & (df_AluMateria['Nota_TP'] >= 7 ) & (df_AluMateria['Nota_TP'] != 11)
                    , 'Condicion'] = 'Promocionado'
#Creo la condicion de Regular basandome en la de Promocionado y agregando algunas condiciones mas.
df_AluMateria.loc[  (df_AluMateria['Condicion'] !='Promocionado')
                    & ((df_AluMateria['Nota_TP'] >= 4) & (df_AluMateria['Nota_2p'] >= 4) & (df_AluMateria['Nota_1p'] >= 4) 
                        & (df_AluMateria['Nota_TP'] !=11) & (df_AluMateria['Nota_2p'] !=11) & (df_AluMateria['Nota_1p'] !=11)) 
                    | ((df_AluMateria['Nota_Recup']!= 11 ) & (df_AluMateria['Nota_Recup']>= 4 ))
                    , 'Condicion'] = "Regular"
#Creo la condicion de libre por descarte de la de Regular y Promocionado.
df_AluMateria.loc[  (df_AluMateria['Condicion'] != "Promocionado") & (df_AluMateria['Condicion'] !='Regular')
                    , 'Condicion'] = 'Libre'

### Recursante

In [19]:
#Uso shift para usar el dato de la fila anterior y lo comparo con el de la fila actual.
#Si son iguales es porque el alumno esta recursando esa materia.
df_AluMateria['Tipo']=np.where(   ((df_AluMateria['Id_Materia'].shift() == df_AluMateria['Id_Materia']))
                                & ((df_AluMateria['Id_Carrera'].shift() == df_AluMateria['Id_Carrera']))
                                & ((df_AluMateria['Id_Alumno'].shift() == df_AluMateria['Id_Alumno']))
                                    ,'Recursante', np.nan)

### Egresado, Cursante, Desertor

In [20]:
df_AluMateria['Estado'] = 'Estado'

In [21]:
df_AluMateria = pd.merge(df_AluMateria,
                        df_Carreras[['Id_Carrera','Cuatrimestres']],
                        how='inner',
                        on=['Id_Carrera'])

In [22]:
#Creo la condicion de Egresado
df_AluMateria.loc[ (df_AluMateria['Cuatrimestre'] == df_AluMateria['Cuatrimestres']), 'Estado'] = 'Egresado' 
#Creo la condicion de Cursando
df_AluMateria.loc[ (df_AluMateria['Estado'] != 'Egresado') &
                    (df_AluMateria['Cuatrimestre'] != df_AluMateria['Cuatrimestres']) &
                    (df_AluMateria['Año'] == 2010) &
                    (df_AluMateria['Semestre']==2), 'Estado'] = 'Cursando'
#Creo la condicion de Desertor
df_AluMateria.loc[(df_AluMateria['Estado']!='Egresado') &
                    (df_AluMateria['Estado']!='Cursando') &
                    (df_AluMateria['Cuatrimestre']!=df_AluMateria['Cuatrimestres'])&
                    (((df_AluMateria['Año']==2010)&(df_AluMateria['Semestre']==1))|
                    ((df_AluMateria['Año']!=2010))),'Estado']='Desertor'

In [23]:
#Utilizo un gruopby para clasificar a los alumnos por sus ID y luego genero un Count de las carreras en las que estuvó.
AlumnosRepetidos = df_ALUCarrera.groupby(['Id_Alumno'])['Id_Carrera'].count().reset_index(name='Cantidad')
AlumnosRepetidos.sort_values(by='Cantidad',ascending=False)

,Id_Alumno,Cantidad
5099,0xD939FAAAF156527DFE6AEF66BD58EC08,3
2831,0x786298BEF8E9AA1F20C3D079463EAB73,2
1420,0x3BB60A9E0F9935677A0427E8D30480A7,2
1417,0x3B8E5503C9C36B2AD3C86F4AB5A83CB7,2
2147,0x5BE27FA218DC751C9E86F6218D4A72F5,2
...,...,...
2020,0x556C2E88464BADDB1645ED5B3AB047A0,1
2019,0x5552F249F93F26722CDF25018BDAC8B6,1
2018,0x554A7492AF4381123F4AE472EA9BEEB3,1
2017,0x554704F29CBB7B9F52E0B82211132C06,1


# Graficas

### Cantidad de Alumnos de cada carrera x Año y semestre

In [24]:
df_Cantidad_Alumnos_Año = df_ALUCarrera.groupby(['Id_Carrera','Año_Ingreso','Semestre_Ingreso'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Cantidad_Alumnos_Año = pd.merge(df_Cantidad_Alumnos_Año,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
px.line(df_Cantidad_Alumnos_Año, x="Año_Ingreso",y='Cantidad', color="Nombre_Corto", title='Cantidad de Alumnos de cada carrera x Año y semestre', facet_col='Semestre_Ingreso')

### Total de Alumnos X Año de la institucion separado en mujeres y varones

In [25]:
px.histogram(df_ALUCarrera, 
            x="Edad", 
            title='Edad de alumnos',
            color='Sexo',
            animation_frame=df_ALUCarrera['Año_Ingreso'].sort_values())

### Total de Alumnos x Carrera separado entre mujeres y varones

In [26]:
df_Sexo_Alumno_xCarrera = df_ALUCarrera.groupby(['Id_Carrera','Sexo'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Sexo_Alumno_xCarrera = pd.merge(df_Sexo_Alumno_xCarrera,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
px.bar(df_Sexo_Alumno_xCarrera, x='Nombre_Corto',y='Cantidad', color='Sexo')

### Total Alumnos x Carrera separado entre Egresado, Desertor y Cursando

In [27]:
df_Estado_Alumno_xCarrera = df_AluMateria.groupby(['Id_Carrera','Año','Estado'])['Id_Alumno'].count().reset_index(name='Cantidad')
df_Estado_Alumno_xCarrera = pd.merge(df_Estado_Alumno_xCarrera,
                        df_Carreras[['Id_Carrera','Nombre_Corto']],
                        how='inner',
                        on=['Id_Carrera'])
px.bar(df_Estado_Alumno_xCarrera, x='Nombre_Corto',y='Cantidad', color='Estado', 
        title='Total Alumnos x Carrera separado entre Egresado y Desertor', 
        animation_frame='Año', 
        barmode='group')

In [28]:
# Cantidad de alumnos por materia a lo largo de los años 

In [29]:
df_AluMateria.head(1)

,Id_Alumno,Id_Carrera,Id_Materia,Semestre,Año,Fecha_Inscrip,Ctro_Ext,Id_Modalidad,Id_Comision,Nota_1p,...,Fecha_2p,Nota_TP,Fecha_TP,Nota_Recup,Fecha_Recup,Cuatrimestre,Condicion,Tipo,Estado,Cuatrimestres
0,0x003014EF4D7D79987F42DE8E3080AEFE,6,115,2,2006.0,2006-08-16 15:02:00,CO,P,7,11.0,...,2006-11-20,11.0,2006-11-29,11.0,2006-11-27,4,Libre,nan,Desertor,6


In [30]:
df_ALUCarrera.head(1)

,Id_Alumno,Id_Carrera,Semestre_Ingreso,Año_Ingreso,Id_Ctro_Ext,Id_Modalidad,Fecha_Ingreso,Activo,Sexo,Fecha_Nac,Edad
0,0x12DA11908B130262F20F714810A49680,6,2,2010,CO,P,2010-07-30,S,M,1991-03-03,19.0
